In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_absolute_error
import xgboost as xgb
from joblib import load, dump
from sklearn.model_selection import train_test_split

import csv

# real

In [2]:
# read data
df = pd.read_csv('1_taipei_purged.csv')

df = df[df['date'] > 202100]

In [3]:
# get 2021 real price
ref = []
for name in df['district'].unique():
    dfsmall = df[df['district'] == name]
    avg = round(dfsmall['avg'].mean(), 2)
    ref.append([name, avg])

In [5]:
# save file
header = ['district', 'real']

with open('check_diff.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(ref)

# pred

In [6]:
# read data and join
df = pd.read_csv('1_taipei_purged.csv')

dfp = pd.read_csv('pop.csv')
dfp = dfp[['year', 'district', 'pop', 'sex_ratio', 'in', 'out', 'growth', 'marry_rate']]

dff = pd.read_csv('AllFinancialCols.csv')
dff = dff[['date', 'load_archi', 'load_house', 'M1B', 'income_rate']]

df['district'] = df['district'].str[3:5]
df['year'] = df['date'] // 100

df = pd.merge(df, dfp, on=['year', 'district'])
df = pd.merge(df, dff, on='date')

df = df.drop(['address', 'latitude', 'longitude', 'style', 'percent', 'management', 'district', 'year'], axis=1)
df.loc[:, 'date'] = df.loc[:, 'date'] % 100 + (df.loc[:, 'date'] // 100 - 2013 ) * 12

In [7]:
# split data into year 2013~2020 & year 2021
train = df[df['date'] <= 96]
test = df[df['date'] > 96]

X_train, y_train = train.drop(['total_price', 'avg'], axis=1), train[['total_price']]
X_test, y_test = test.drop(['total_price', 'avg'], axis=1), test[['total_price']]

In [8]:
# train
model = load('house.joblib')

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.05, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.25, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=13, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.3, reg_lambda=0.09, ...)

In [9]:
# get predict price
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

pred = pd.DataFrame(y_pred, columns = ['pred'])

In [11]:
# makd pred for making check_diff
df = pd.read_csv('1_taipei_purged.csv')

dfp = pd.read_csv('pop.csv')
dfp = dfp[['year', 'district', 'pop', 'sex_ratio', 'in', 'out', 'growth', 'marry_rate']]

dff = pd.read_csv('AllFinancialCols.csv')
dff = dff[['date', 'load_archi', 'load_house', 'M1B', 'income_rate']]

df['district'] = df['district'].str[3:5]
df['year'] = df['date'] // 100

df = pd.merge(df, dfp, on=['year', 'district'])
df = pd.merge(df, dff, on='date')

# df = df.drop(['address', 'latitude', 'longitude', 'style', 'percent', 'management', 'district', 'year'], axis=1)


test = df[df['date'] > 202100]

test['pred'] = pred

test['avg_pred'] = test['pred'] / test['size']

/tmp/ipykernel_3250/2662979869.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = pred
/tmp/ipykernel_3250/2662979869.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['avg_pred'] = test['pred'] / test['size']


In [12]:
# show pred
ref = []
for name in test['district'].unique():
    dfsmall = test[test['district'] == name]
    avg = round(dfsmall['avg_pred'].mean(), 2)
    ref.append(avg)
ref

[62.9,
 62.04,
 63.01,
 56.38,
 47.59,
 54.17,
 49.65,
 51.78,
 67.96,
 53.43,
 63.89,
 50.92,
 25.03,
 10.01,
 38.17,
 41.47,
 24.69,
 20.8,
 35.52,
 39.92,
 39.38,
 43.9,
 27.7,
 26.2,
 43.44,
 35.59,
 23.87,
 26.14,
 24.15,
 10.39,
 36.42,
 16.09,
 23.12,
 9.19,
 6.34,
 7.89,
 7.3,
 13.55,
 19.2]

# merge pred

In [13]:
# reload data
df = pd.read_csv('1_taipei_purged.csv')

dfp = pd.read_csv('pop.csv')
dfp = dfp[['year', 'district', 'pop', 'sex_ratio', 'in', 'out', 'growth', 'marry_rate']]

dff = pd.read_csv('AllFinancialCols.csv')
dff = dff[['date', 'load_archi', 'load_house', 'M1B', 'income_rate']]

df['district'] = df['district'].str[3:5]
df['year'] = df['date'] // 100

df = pd.merge(df, dfp, on=['year', 'district'])
df = pd.merge(df, dff, on='date')

In [14]:
# join predict price of 2021
df['pred'] = pred

df['avg_pred'] = df['pred'] / df['size']

df = df[df['date'] > 202100]


In [15]:
# show check_diff
pred = pd.DataFrame(ref, columns = ['pred'])

table = pd.read_csv('check_diff.csv')

table['pred'] = pred
table['diff'] = table['real'] - table['pred']
table['diff'] = table['diff'].round(2)
table

,district,real,pred,diff
0,台北市中山區,63.38,62.90,0.48
1,台北市中正區,66.19,62.04,4.15
2,台北市信義區,63.64,63.01,0.63
3,台北市內湖區,56.35,56.38,-0.03
4,台北市北投區,46.47,47.59,-1.12
5,台北市南港區,57.49,54.17,3.32
6,台北市士林區,53.91,49.65,4.26
7,台北市大同區,54.93,51.78,3.15
8,台北市大安區,70.15,67.96,2.19
9,台北市文山區,49.36,53.43,-4.07


In [16]:
table.to_csv('check_diff.csv', index=False)